In [2]:
# MULTI-MODAL TSP TRAVEL PLANNER

import requests
import googlemaps
import time
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2 # OR-Tools TSP solver

from dotenv import load_dotenv
import os

load_dotenv()
maps_api_key = os.getenv("MAPS_API_KEY")

In [3]:
API_KEY_GOOGLE = maps_api_key
API_KEY_FLIGHT = 'YOUR_FLIGHTAPI_IO_KEY'
gmaps = googlemaps.Client(key=API_KEY_GOOGLE)

locations = ["Vail, CO", "Beaver Creek, CO", "Breckenridge, CO", "Keystone, CO", "A-Basin, CO", "Eldora, CO", "Canyons, UT", "Heavenly, CA", "Northstar, CA", "Kirkwood, CA", "Afton Alps, MN", "Mt. Brighton, MI", "Verbier, Switzerland", "Ski Arlberg, Austria", "The 3 Valleys, France"]
nearest_airport = {
    "Vail, CO": "DEN",
    "Beaver Creek, CO": "DEN",
    "Breckenridge, CO": "DEN",
    "Keystone, CO": "DEN", 
    "A-Basin, CO": "DEN", 
    "Eldora, CO": "DEN", 
    "Canyons, UT": "SLC",
    "Heavenly, CA": "RNO",
    "Northstar, CA": "RNO",
    "Kirkwood, CA": "RNO",
    "Afton Alps, MN": "MSP",
    "Mt. Brighton, MI": "DTW",
    "Verbier, Switzerland": "GVA",
    "Ski Arlberg, Austria": "INNS",
    "The 3 Valleys, France": "GVA"
}


ValueError: Must provide API key or enterprise credentials when creating client.

In [ ]:
# STEP 1: BUILD COST MATRIX

def get_drive_info(origin, destination):
    try:
        res = gmaps.directions(origin, destination, mode="driving")
        if res:
            leg = res[0]['legs'][0]
            distance_mi = leg['distance']['value'] * 0.000621371
            duration_hr = leg['duration']['value'] / 3600
            return distance_mi, duration_hr
    except:
        pass
    return None, None

def get_flight_cost(orig_airport, dest_airport, api_key):
    url = f"https://www.flightapi.io/onewaytrip/{api_key}/{orig_airport}/{dest_airport}/2024-08-10/1/0/0/Economy/USD"
    try:
        r = requests.get(url)
        data = r.json()
        prices = [trip['price'] for trip in data.get('trips', []) if 'price' in trip]
        return min(prices) if prices else None
    except:
        return None

def build_cost_matrix(locations):
    n = len(locations)
    cost_matrix = np.full((n, n), np.inf)
    mode_matrix = [['' for _ in range(n)] for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            A, B = locations[i], locations[j]

            # DRIVE
            dist_drive, _ = get_drive_info(A, B)
            cost_drive = dist_drive * 0.5 if dist_drive else np.inf

            # FLY
            airport_A, airport_B = nearest_airport[A], nearest_airport[B]
            drive1, _ = get_drive_info(A, airport_A)
            drive2, _ = get_drive_info(airport_B, B)
            flight_price = get_flight_cost(airport_A, airport_B, API_KEY_FLIGHT)
            
            if None not in [drive1, drive2, flight_price]:
                cost_fly = flight_price + 0.5 * (drive1 + drive2)
            else:
                cost_fly = np.inf

            # CHOOSE BEST MODE
            if cost_drive < cost_fly:
                cost_matrix[i][j] = cost_drive
                mode_matrix[i][j] = "drive"
            else:
                cost_matrix[i][j] = cost_fly
                mode_matrix[i][j] = "fly"

            time.sleep(1)

    return cost_matrix, mode_matrix

In [ ]:
# STEP 2: SOLVE TSP

def solve_tsp(cost_matrix):
    n = len(cost_matrix)
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def cost_callback(from_idx, to_idx):
        from_node = manager.IndexToNode(from_idx)
        to_node = manager.IndexToNode(to_idx)
        return int(cost_matrix[from_node][to_node] * 100)

    transit_callback_idx = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_idx)

    params = pywrapcp.DefaultRoutingSearchParameters()
    params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(params)
    if not solution:
        raise Exception("TSP solution not found")

    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))
    return route

In [ ]:
# STEP 3: EVALUATE TRIP

def summarize_trip(route, locations, cost_matrix, mode_matrix):
    total_cost = 0
    for i in range(len(route)-1):
        a, b = route[i], route[i+1]
        origin, dest = locations[a], locations[b]
        cost = cost_matrix[a][b]
        mode = mode_matrix[a][b]
        total_cost += cost
        print(f"{origin} → {dest}: ${cost:.2f} via {mode}")
    print(f"\nTotal trip cost: ${total_cost:.2f}")

# MAIN FLOW
cost_matrix, mode_matrix = build_cost_matrix(locations)
route = solve_tsp(cost_matrix)
summarize_trip(route, locations, cost_matrix, mode_matrix)


* Google Maps API Docs: https://developers.google.com/maps/documentation/directions/start

* FlightAPI.io Docs: https://www.flightapi.io/docs

* Google OR-Tools for Routing (TSP): https://developers.google.com/optimization/routing

* Haversine Formula (for backup distance calculations): https://en.wikipedia.org/wiki/Haversine_formula

* Python Requests Library: https://docs.python-requests.org/